In [ ]:
!pip install Sastrawi

# Import Libraries

In [ ]:
import re
import matplotlib.pyplot as plt
import string
from nltk.corpus import stopwords
import nltk
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer
from nltk.tokenize.treebank import TreebankWordDetokenizer
from collections import Counter
from wordcloud import WordCloud
from nltk.corpus import stopwords
import nltk
nltk.download('punkt')
from gensim.utils import simple_preprocess
from nltk.corpus import stopwords
import gensim
from sklearn.model_selection import train_test_split
import spacy
import pickle
import warnings
warnings.filterwarnings('ignore')
import seaborn as sns
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt 
import tensorflow as tf
import keras
import numpy as np
import pandas as pd
import string


from nltk.tokenize import word_tokenize
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory 
print('Done')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
Done


# Data Import

In [ ]:
data = pd.read_csv('dataset_tokopedia.csv')

# Data Preprocessing and Data Cleaning

In [ ]:
data.head(15)

,Unnamed: 0,text,rating,category,product_name,product_id,sold,shop_id,product_url
0,95,barang yg dikirim tidak sesuai pesanan,1,pertukangan,Isi Staples Tembak 8 mm best guard,133507638,545,1461393,https://www.tokopedia.com/timurjaya46/isi-stap...
1,117,"Php, bilang isi ada diseskripsi pas dipesen be...",1,pertukangan,steples tembak / staples tembak kenmaster PROM...,88842566,45,1102298,https://www.tokopedia.com/cahayabelawa/steples...
2,188,Beli staples gak jual isinya... hanya sekali p...,1,pertukangan,Staple Gun / Staples Tembak / Staples Jok / He...,209226141,171,580197,https://www.tokopedia.com/hmhhardware/staple-g...
3,255,Sebaiknya kalau mau ngirim barang...diperiksa ...,1,pertukangan,Isi / Refill Staples / Stapler Tembak / Steple...,55221811,67,452241,https://www.tokopedia.com/cgrtools/isi-refill-...
4,826,"barang datang cacat ,gak bisa di gunakan \nhar...",1,pertukangan,Staple Staples Steples Gun Tembak Kenmaster Gu...,213653744,122,1461393,https://www.tokopedia.com/timurjaya46/staple-s...
5,865,Kecewa baru dipakey isiya kluar sampai 2 3 bua...,1,pertukangan,STAPLE GUN MOLLAR 3 WAY TACKER - STAPLES JOK T...,288306960,148,1477109,https://www.tokopedia.com/juraganperkakas/stap...
6,1015,"Hasil staples tidak rapat pada bidang, mungkin...",1,pertukangan,Staples Gun Tacker Mollar 3 in 1 / Staple Jok ...,221298191,408,1114588,https://www.tokopedia.com/indahjayatools/stapl...
7,1074,Barangnya rusak..kecewa berat,1,pertukangan,STAPLE GUN SERBAGUNA MOLLAR STAPLES TEMBAK BON...,225536821,85,537844,https://www.tokopedia.com/samudrabauteknik/sta...
8,1244,Beli 5 tdk bs dipakai semua,1,pertukangan,Staple/Stapel/Straples/Staples Nail &amp; paku...,78773158,207,1371033,https://www.tokopedia.com/totalteknik/staplest...
9,1246,"Straples nya gk bs dipake, udh dicoba berkali...",1,pertukangan,Staple/Stapel/Straples/Staples Nail &amp; paku...,78773158,207,1371033,https://www.tokopedia.com/totalteknik/staplest...


In [ ]:
#Mengecek jumlah data null
len(data) - len(data.dropna())

14

In [ ]:
#Mengecek jumlah data
len(data)

40607

In [ ]:
#Membuang data null
data = data.dropna()
len(data)
data.head()

,Unnamed: 0,text,rating,category,product_name,product_id,sold,shop_id,product_url
0,95,barang yg dikirim tidak sesuai pesanan,1,pertukangan,Isi Staples Tembak 8 mm best guard,133507638,545,1461393,https://www.tokopedia.com/timurjaya46/isi-stap...
1,117,"Php, bilang isi ada diseskripsi pas dipesen be...",1,pertukangan,steples tembak / staples tembak kenmaster PROM...,88842566,45,1102298,https://www.tokopedia.com/cahayabelawa/steples...
2,188,Beli staples gak jual isinya... hanya sekali p...,1,pertukangan,Staple Gun / Staples Tembak / Staples Jok / He...,209226141,171,580197,https://www.tokopedia.com/hmhhardware/staple-g...
3,255,Sebaiknya kalau mau ngirim barang...diperiksa ...,1,pertukangan,Isi / Refill Staples / Stapler Tembak / Steple...,55221811,67,452241,https://www.tokopedia.com/cgrtools/isi-refill-...
4,826,"barang datang cacat ,gak bisa di gunakan \nhar...",1,pertukangan,Staple Staples Steples Gun Tembak Kenmaster Gu...,213653744,122,1461393,https://www.tokopedia.com/timurjaya46/staple-s...


In [ ]:
#Drop column yang tidak digunakan
data = data.drop (['Unnamed: 0','category','product_id','sold', 'shop_id', 'product_url'], axis = 1)

In [ ]:
#Menambahkan kolom sentimen sesuai rating review
def sentiment(n):
    if n <= 3:
      return 0
    else:
      return 1
data['sentiment'] = data['rating'].apply(sentiment)

In [ ]:
data.groupby(by='sentiment').count()

,text,rating,product_name
sentiment,,,
0,2750,2750,2750
1,37843,37843,37843


In [ ]:
data = data.iloc[:6000,:]

In [ ]:
data.groupby(by='sentiment').count()

,text,rating,product_name
sentiment,,,
0,2750,2750,2750
1,3250,3250,3250


In [ ]:
print(data['text'])

0                   barang yg dikirim tidak sesuai pesanan
1        Php, bilang isi ada diseskripsi pas dipesen be...
2        Beli staples gak jual isinya... hanya sekali p...
3        Sebaiknya kalau mau ngirim barang...diperiksa ...
4        barang datang cacat ,gak bisa di gunakan \nhar...
                               ...                        
40602      Barang sudah kami terima ya gan.terimakasih gan
40603        bagus barangnya.lolos quality cek.mantep dech
40604             thaaaanksss barang bagus cepat sampai :)
40605     fast respon dan barang sesuai dgn yg di iklankan
40606    Mantab, pengiriman sangat cepat, barang pking ...
Name: text, Length: 40593, dtype: object


In [ ]:
data['review'] = data['text'].str.lower()

In [ ]:
def remove_special_character(text):
    text = text.replace('\\t'," ").replace('\\n'," ").replace('\\u'," ").replace('\\',"")
    text = text.encode('ascii', 'replace').decode('ascii')
    text = ' '.join(re.sub("([@#][A-Za-z0-9]+)|(\w+:\/\/\S+)"," ", text).split())
    return text.replace("http://", " ").replace("https://", " ")
data['review'] = data['review'].apply(remove_special_character)   

In [ ]:
def remove_number(text):
    return  re.sub(r"\d+", "", text)


data['review'] = data['review'].apply(remove_number)

In [ ]:
def remove_punctuation(text):
    return text.translate(str.maketrans("","",string.punctuation))

data['review'] = data['review'].apply(remove_punctuation)

In [ ]:
def remove_whitespace(text):
    txt = text.strip()
    return re.sub('\s+',' ',txt)
data['review'] = data['review'].apply(remove_whitespace)

In [ ]:
def remove_singl_char(text):
    return re.sub(r"\b[a-zA-Z]\b", "", text)
data['review'] = data['review'].apply(remove_singl_char)

In [ ]:
factory = StopWordRemoverFactory()
stopword = factory.create_stop_word_remover()

def stopwords_removal(text):
  return stopword.remove(text)

data['review'] = data['review'].apply(stopwords_removal) 

In [ ]:
print(data['review'])

0                        barang yg dikirim sesuai pesanan
1       php bilang isi diseskripsi pas dipesen berbeda...
2         beli staples gak jual isinya sekali pake ckckck
3       sebaiknya kalau mau ngirim barangdiperiksa dul...
4       barang datang cacat gak gunakan harusnya sblum...
                              ...                        
5998     usefull pemakaian standard cukup baik dikelasnya
5999                  terimakasih maju jaya aktifkan jaya
6000                     barang nyampe gan sesuai pesanan
6001                            bagus aja brg pabrik juga
6002                                good barang berfungsi
Name: review, Length: 6000, dtype: object


In [ ]:
#Mengubah series panda to list
temp = []
data_to_list = data['review'].values.tolist()
for i in range(len(data_to_list)):
    temp.append(data_to_list[i])

In [ ]:
data_list = np.array(data_to_list)

In [ ]:
#Label Encoding
labels = np.array(data['sentiment'])
y = []
for i in range(len(labels)):
    if labels[i] == 0:
        y.append(0)
    if labels[i] == 1:
        y.append(1)

y = np.array(y)
labels = tf.keras.utils.to_categorical(y, 3, dtype="float32")
del y

In [ ]:
#Data Tokenizing
from keras.models import Sequential
from keras import layers
from tensorflow.keras.optimizers import RMSprop,Adam
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras import regularizers
from keras import backend as K
from keras.callbacks import ModelCheckpoint
max_words = 5000
max_len = 200

tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(data_list)
sequences = tokenizer.texts_to_sequences(data_list)
review = pad_sequences(sequences, maxlen=max_len)
print(review)

[[   0    0    0 ...   30    2   14]
 [   0    0    0 ...  394   22  171]
 [   0    0    0 ...   82   96 1832]
 ...
 [   0    0    0 ...   13    2   14]
 [   0    0    0 ...   76 1738  330]
 [   0    0    0 ...   84    1   48]]


In [ ]:
print(labels)

[[1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 ...
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]]


In [ ]:
#Data Train Test Split
X_train, X_test, y_train, y_test = train_test_split(review,labels, random_state=0)
print (len(X_train),len(X_test),len(y_train),len(y_test))

4500 1500 4500 1500


# Model 1-1

In [ ]:
model1 = Sequential()
model1.add(layers.Embedding(max_words, 40, input_length=max_len))
model1.add(layers.LSTM(5,dropout=0.6))
model1.add(layers.Dense(3,activation='softmax'))
model1.compile(optimizer='rmsprop',loss='categorical_crossentropy', metrics=['accuracy'])

checkpoint1 = ModelCheckpoint("model1.hdf5", monitor='val_loss', verbose=2,save_best_only=True, mode='auto', period=1,save_weights_only=False)
history = model1.fit(X_train, y_train, epochs=50,validation_data=(X_test, y_test),callbacks=[checkpoint1])

Epoch 1/50
141/141 [==============================] - ETA: 0s - loss: 0.8026 - accuracy: 0.5267
Epoch 00001: val_loss improved from inf to 0.70823, saving model to model1.hdf5
141/141 [==============================] - 23s 148ms/step - loss: 0.8026 - accuracy: 0.5267 - val_loss: 0.7082 - val_accuracy: 0.6073
Epoch 2/50
141/141 [==============================] - ETA: 0s - loss: 0.6839 - accuracy: 0.6364
Epoch 00002: val_loss improved from 0.70823 to 0.64631, saving model to model1.hdf5
141/141 [==============================] - 20s 144ms/step - loss: 0.6839 - accuracy: 0.6364 - val_loss: 0.6463 - val_accuracy: 0.6767
Epoch 3/50
141/141 [==============================] - ETA: 0s - loss: 0.6306 - accuracy: 0.6731
Epoch 00003: val_loss improved from 0.64631 to 0.60682, saving model to model1.hdf5
141/141 [==============================] - 10s 68ms/step - loss: 0.6306 - accuracy: 0.6731 - val_loss: 0.6068 - val_accuracy: 0.6940
Epoch 4/50
141/141 [==============================] - ETA: 0s -

In [ ]:
model1.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 200, 40)           200000    
                                                                 
 lstm_1 (LSTM)               (None, 5)                 920       
                                                                 
 dense_1 (Dense)             (None, 3)                 18        
                                                                 
Total params: 200,938
Trainable params: 200,938
Non-trainable params: 0
_________________________________________________________________


In [ ]:
best_model_1 = keras.models.load_model("model1.hdf5")

In [ ]:
test_loss, test_acc, = best_model_1.evaluate(X_test, y_test, verbose=2)

47/47 - 1s - loss: 0.5626 - accuracy: 0.7213 - 895ms/epoch - 19ms/step


In [ ]:
predictions1 = best_model_1.predict(X_test)

In [ ]:
from sklearn.metrics import confusion_matrix
matrix = confusion_matrix(y_test.argmax(axis=1), np.around(predictions1.argmax(axis=1)))
print(matrix)  #tn = 398,fp = 262,fn = 147,tp = 693 

[[358 302]
 [116 724]]


In [ ]:
from sklearn.metrics import precision_recall_fscore_support as eval

In [ ]:
precision, recall, fscore, support = eval(y_test.argmax(axis=1), predictions1.argmax(axis=1))
print('precision (Negative, Positive):', precision)
print('recall (Negative, Positive):', recall)
print('fscore (Negative, Positive):', fscore)

precision (Negative, Positive): [0.75527426 0.70565302]
recall (Negative, Positive): [0.54242424 0.86190476]
fscore (Negative, Positive): [0.6313933  0.77599143]


# Model 2-1

In [ ]:
model2 = Sequential()
model2.add(layers.Embedding(max_words, 40, input_length=max_len))
model2.add(layers.LSTM(10,dropout=0.6))
model2.add(layers.Dense(3,activation='softmax'))
model2.compile(optimizer='rmsprop',loss='categorical_crossentropy', metrics=['accuracy'])

checkpoint2 = ModelCheckpoint("model2.hdf5", monitor='val_loss', verbose=2,save_best_only=True, mode='auto', period=1,save_weights_only=False)
history = model2.fit(X_train, y_train, epochs=50,validation_data=(X_test, y_test),callbacks=[checkpoint2])

Epoch 1/50
141/141 [==============================] - ETA: 0s - loss: 0.7613 - accuracy: 0.5916
Epoch 00001: val_loss improved from inf to 0.64653, saving model to model2.hdf5
141/141 [==============================] - 12s 69ms/step - loss: 0.7613 - accuracy: 0.5916 - val_loss: 0.6465 - val_accuracy: 0.6560
Epoch 2/50
141/141 [==============================] - ETA: 0s - loss: 0.6335 - accuracy: 0.6556
Epoch 00002: val_loss improved from 0.64653 to 0.59750, saving model to model2.hdf5
141/141 [==============================] - 9s 65ms/step - loss: 0.6335 - accuracy: 0.6556 - val_loss: 0.5975 - val_accuracy: 0.6920
Epoch 3/50
141/141 [==============================] - ETA: 0s - loss: 0.5886 - accuracy: 0.6947
Epoch 00003: val_loss improved from 0.59750 to 0.57768, saving model to model2.hdf5
141/141 [==============================] - 9s 65ms/step - loss: 0.5886 - accuracy: 0.6947 - val_loss: 0.5777 - val_accuracy: 0.7153
Epoch 4/50
141/141 [==============================] - ETA: 0s - los

In [ ]:
model2.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_2 (Embedding)     (None, 200, 40)           200000    
                                                                 
 lstm_2 (LSTM)               (None, 10)                2040      
                                                                 
 dense_2 (Dense)             (None, 3)                 33        
                                                                 
Total params: 202,073
Trainable params: 202,073
Non-trainable params: 0
_________________________________________________________________


In [ ]:
best_model_2 = keras.models.load_model("model2.hdf5")

In [ ]:
test_loss, test_acc = best_model_2.evaluate(X_test, y_test, verbose=2)

47/47 - 1s - loss: 0.5578 - accuracy: 0.7200 - 868ms/epoch - 18ms/step


In [ ]:
predictions2 = best_model_2.predict(X_test)

In [ ]:
from sklearn.metrics import confusion_matrix
matrix = confusion_matrix(y_test.argmax(axis=1), np.around(predictions2.argmax(axis=1)))
print(matrix)  #tn = 398,fp = 262,fn = 147,tp = 693 

[[356 304]
 [116 724]]


In [ ]:
from sklearn.metrics import precision_recall_fscore_support as eval

In [ ]:
precision, recall, fscore, support = eval(y_test.argmax(axis=1), predictions2.argmax(axis=1))
print('precision (Negative, Positive):', precision)
print('recall (Negative, Positive):', recall)
print('fscore (Negative, Positive):', fscore)

precision (Negative, Positive): [0.75423729 0.70428016]
recall (Negative, Positive): [0.53939394 0.86190476]
fscore (Negative, Positive): [0.62897527 0.7751606 ]


# Model 3-1

In [ ]:
model3 = Sequential()
model3.add(layers.Embedding(max_words, 40, input_length=max_len))
model3.add(layers.LSTM(15,dropout=0.6))
model3.add(layers.Dense(3,activation='softmax'))
model3.compile(optimizer='rmsprop',loss='categorical_crossentropy', metrics=['accuracy'])

checkpoint3 = ModelCheckpoint("model3.hdf5", monitor='val_loss', verbose=2,save_best_only=True, mode='auto', period=1,save_weights_only=False)
history = model3.fit(X_train, y_train, epochs=50,validation_data=(X_test, y_test),callbacks=[checkpoint3])

Epoch 1/50
141/141 [==============================] - ETA: 0s - loss: 0.7256 - accuracy: 0.5593
Epoch 00001: val_loss improved from inf to 0.65926, saving model to model3.hdf5
141/141 [==============================] - 12s 71ms/step - loss: 0.7256 - accuracy: 0.5593 - val_loss: 0.6593 - val_accuracy: 0.6413
Epoch 2/50
141/141 [==============================] - ETA: 0s - loss: 0.6304 - accuracy: 0.6607
Epoch 00002: val_loss improved from 0.65926 to 0.59811, saving model to model3.hdf5
141/141 [==============================] - 10s 68ms/step - loss: 0.6304 - accuracy: 0.6607 - val_loss: 0.5981 - val_accuracy: 0.7027
Epoch 3/50
141/141 [==============================] - ETA: 0s - loss: 0.5890 - accuracy: 0.6933
Epoch 00003: val_loss improved from 0.59811 to 0.57007, saving model to model3.hdf5
141/141 [==============================] - 10s 69ms/step - loss: 0.5890 - accuracy: 0.6933 - val_loss: 0.5701 - val_accuracy: 0.7207
Epoch 4/50
141/141 [==============================] - ETA: 0s - l

In [ ]:
model3.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_3 (Embedding)     (None, 200, 40)           200000    
                                                                 
 lstm_3 (LSTM)               (None, 15)                3360      
                                                                 
 dense_3 (Dense)             (None, 3)                 48        
                                                                 
Total params: 203,408
Trainable params: 203,408
Non-trainable params: 0
_________________________________________________________________


In [ ]:
best_model_3 = keras.models.load_model("model3.hdf5")

In [ ]:
test_loss, test_acc = best_model_3.evaluate(X_test, y_test, verbose=2)

47/47 - 1s - loss: 0.5601 - accuracy: 0.7213 - 914ms/epoch - 19ms/step


In [ ]:
predictions3 = best_model_3.predict(X_test)

In [ ]:
from sklearn.metrics import confusion_matrix
matrix = confusion_matrix(y_test.argmax(axis=1), np.around(predictions3.argmax(axis=1)))
print(matrix)

[[369 291]
 [127 713]]


In [ ]:
from sklearn.metrics import precision_recall_fscore_support as eval

In [ ]:
precision, recall, fscore, support = eval(y_test.argmax(axis=1), predictions3.argmax(axis=1))
print('precision (Negative, Positive):', precision)
print('recall (Negative, Positive):', recall)
print('fscore (Negative, Positive):', fscore)

precision (Negative, Positive): [0.74395161 0.71015936]
recall (Negative, Positive): [0.55909091 0.84880952]
fscore (Negative, Positive): [0.6384083  0.77331887]


# Model 1-2

In [ ]:
model4 = Sequential()
model4.add(layers.Embedding(max_words, 40, input_length=max_len))
model4.add(layers.LSTM(5,dropout=0.6))
model4.add(layers.Dense(3,activation='softmax'))
model4.compile(optimizer='adam',loss='categorical_crossentropy', metrics=['accuracy'])

checkpoint4 = ModelCheckpoint("model4.hdf5", monitor='val_loss', verbose=2,save_best_only=True, mode='auto', period=1,save_weights_only=False)
history = model4.fit(X_train, y_train, epochs=50,validation_data=(X_test, y_test),callbacks=[checkpoint4])

Epoch 1/50
141/141 [==============================] - ETA: 0s - loss: 0.8936 - accuracy: 0.5913
Epoch 00001: val_loss improved from inf to 0.72024, saving model to model4.hdf5
141/141 [==============================] - 11s 68ms/step - loss: 0.8936 - accuracy: 0.5913 - val_loss: 0.7202 - val_accuracy: 0.6560
Epoch 2/50
141/141 [==============================] - ETA: 0s - loss: 0.6758 - accuracy: 0.6673
Epoch 00002: val_loss improved from 0.72024 to 0.63419, saving model to model4.hdf5
141/141 [==============================] - 9s 65ms/step - loss: 0.6758 - accuracy: 0.6673 - val_loss: 0.6342 - val_accuracy: 0.6953
Epoch 3/50
141/141 [==============================] - ETA: 0s - loss: 0.5859 - accuracy: 0.7309
Epoch 00003: val_loss improved from 0.63419 to 0.57849, saving model to model4.hdf5
141/141 [==============================] - 9s 65ms/step - loss: 0.5859 - accuracy: 0.7309 - val_loss: 0.5785 - val_accuracy: 0.7313
Epoch 4/50
141/141 [==============================] - ETA: 0s - los

In [ ]:
model4.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_4 (Embedding)     (None, 200, 40)           200000    
                                                                 
 lstm_4 (LSTM)               (None, 5)                 920       
                                                                 
 dense_4 (Dense)             (None, 3)                 18        
                                                                 
Total params: 200,938
Trainable params: 200,938
Non-trainable params: 0
_________________________________________________________________


In [ ]:
best_model_4 = keras.models.load_model("model4.hdf5")

In [ ]:
test_loss, test_acc = best_model_2.evaluate(X_test, y_test, verbose=2)

47/47 - 0s - loss: 0.5578 - accuracy: 0.7200 - 457ms/epoch - 10ms/step


In [ ]:
predictions4 = best_model_4.predict(X_test)

In [ ]:
from sklearn.metrics import confusion_matrix
matrix = confusion_matrix(y_test.argmax(axis=1), np.around(predictions4.argmax(axis=1)))
print(matrix)

[[375 285]
 [125 715]]


In [ ]:
from sklearn.metrics import precision_recall_fscore_support as eval

In [ ]:
precision, recall, fscore, support = eval(y_test.argmax(axis=1), predictions4.argmax(axis=1))
print('precision (Negative, Positive):', precision)
print('recall (Negative, Positive):', recall)
print('fscore (Negative, Positive):', fscore)

precision (Negative, Positive): [0.75  0.715]
recall (Negative, Positive): [0.56818182 0.85119048]
fscore (Negative, Positive): [0.64655172 0.77717391]


# Model 2-2

In [ ]:
model5 = Sequential()
model5.add(layers.Embedding(max_words, 40, input_length=max_len))
model5.add(layers.LSTM(10,dropout=0.6))
model5.add(layers.Dense(3,activation='softmax'))
model5.compile(optimizer='adam',loss='categorical_crossentropy', metrics=['accuracy'])

checkpoint5 = ModelCheckpoint("model5.hdf5", monitor='val_loss', verbose=2,save_best_only=True, mode='auto', period=1,save_weights_only=False)
history = model5.fit(X_train, y_train, epochs=50,validation_data=(X_test, y_test),callbacks=[checkpoint5])

Epoch 1/50
141/141 [==============================] - ETA: 0s - loss: 0.7996 - accuracy: 0.5800
Epoch 00001: val_loss improved from inf to 0.64803, saving model to model5.hdf5
141/141 [==============================] - 11s 68ms/step - loss: 0.7996 - accuracy: 0.5800 - val_loss: 0.6480 - val_accuracy: 0.6500
Epoch 2/50
141/141 [==============================] - ETA: 0s - loss: 0.6266 - accuracy: 0.6669
Epoch 00002: val_loss improved from 0.64803 to 0.58927, saving model to model5.hdf5
141/141 [==============================] - 9s 66ms/step - loss: 0.6266 - accuracy: 0.6669 - val_loss: 0.5893 - val_accuracy: 0.7113
Epoch 3/50
141/141 [==============================] - ETA: 0s - loss: 0.5592 - accuracy: 0.7296
Epoch 00003: val_loss improved from 0.58927 to 0.56913, saving model to model5.hdf5
141/141 [==============================] - 9s 66ms/step - loss: 0.5592 - accuracy: 0.7296 - val_loss: 0.5691 - val_accuracy: 0.7107
Epoch 4/50
141/141 [==============================] - ETA: 0s - los

In [ ]:
model5.summary()

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_5 (Embedding)     (None, 200, 40)           200000    
                                                                 
 lstm_5 (LSTM)               (None, 10)                2040      
                                                                 
 dense_5 (Dense)             (None, 3)                 33        
                                                                 
Total params: 202,073
Trainable params: 202,073
Non-trainable params: 0
_________________________________________________________________


In [ ]:
best_model_5 = keras.models.load_model("model5.hdf5")

In [ ]:
test_loss, test_acc = best_model_5.evaluate(X_test, y_test, verbose=2)

47/47 - 1s - loss: 0.5550 - accuracy: 0.7287 - 924ms/epoch - 20ms/step


In [ ]:
predictions5 = best_model_5.predict(X_test)

In [ ]:
from sklearn.metrics import confusion_matrix
matrix = confusion_matrix(y_test.argmax(axis=1), np.around(predictions5.argmax(axis=1)))
print(matrix)  #tn = 398,fp = 262,fn = 147,tp = 693 

[[402 258]
 [149 691]]


In [ ]:
from sklearn.metrics import precision_recall_fscore_support as eval

In [ ]:
precision, recall, fscore, support = eval(y_test.argmax(axis=1), predictions5.argmax(axis=1))
print('precision (Negative, Positive):', precision)
print('recall (Negative, Positive):', recall)
print('fscore (Negative, Positive):', fscore)

precision (Negative, Positive): [0.72958258 0.72813488]
recall (Negative, Positive): [0.60909091 0.82261905]
fscore (Negative, Positive): [0.66391412 0.7724986 ]


# Model 3-2

In [ ]:
model5 = Sequential()
model5.add(layers.Embedding(max_words, 40, input_length=max_len))
model5.add(layers.LSTM(15,dropout=0.6))
model5.add(layers.Dense(3,activation='softmax'))
model5.compile(optimizer='adam',loss='categorical_crossentropy', metrics=['accuracy'])

checkpoint5 = ModelCheckpoint("model5.hdf5", monitor='val_loss', verbose=2,save_best_only=True, mode='auto', period=1,save_weights_only=False)
history = model5.fit(X_train, y_train, epochs=50,validation_data=(X_test, y_test),callbacks=[checkpoint5])

Epoch 1/50
141/141 [==============================] - ETA: 0s - loss: 0.7587 - accuracy: 0.5949
Epoch 00001: val_loss improved from inf to 0.63192, saving model to model5.hdf5
141/141 [==============================] - 12s 75ms/step - loss: 0.7587 - accuracy: 0.5949 - val_loss: 0.6319 - val_accuracy: 0.6613
Epoch 2/50
141/141 [==============================] - ETA: 0s - loss: 0.6105 - accuracy: 0.6791
Epoch 00002: val_loss improved from 0.63192 to 0.58587, saving model to model5.hdf5
141/141 [==============================] - 10s 71ms/step - loss: 0.6105 - accuracy: 0.6791 - val_loss: 0.5859 - val_accuracy: 0.7213
Epoch 3/50
141/141 [==============================] - ETA: 0s - loss: 0.5388 - accuracy: 0.7387
Epoch 00003: val_loss improved from 0.58587 to 0.56729, saving model to model5.hdf5
141/141 [==============================] - 10s 71ms/step - loss: 0.5388 - accuracy: 0.7387 - val_loss: 0.5673 - val_accuracy: 0.7293
Epoch 4/50
141/141 [==============================] - ETA: 0s - l

In [ ]:
model5.summary()

Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_6 (Embedding)     (None, 200, 40)           200000    
                                                                 
 lstm_6 (LSTM)               (None, 15)                3360      
                                                                 
 dense_6 (Dense)             (None, 3)                 48        
                                                                 
Total params: 203,408
Trainable params: 203,408
Non-trainable params: 0
_________________________________________________________________


In [ ]:
best_model_5 = keras.models.load_model("model5.hdf5")

In [ ]:
test_loss, test_acc = best_model_5.evaluate(X_test, y_test, verbose=2)

47/47 - 1s - loss: 0.5659 - accuracy: 0.7233 - 897ms/epoch - 19ms/step


In [ ]:
predictions5 = best_model_5.predict(X_test)

In [ ]:
from sklearn.metrics import confusion_matrix
matrix = confusion_matrix(y_test.argmax(axis=1), np.around(predictions5.argmax(axis=1)))
print(matrix)

[[464 196]
 [219 621]]


In [ ]:
from sklearn.metrics import precision_recall_fscore_support as eval

In [ ]:
precision, recall, fscore, support = eval(y_test.argmax(axis=1), predictions5.argmax(axis=1))
print('precision (Negative, Positive):', precision)
print('recall (Negative, Positive):', recall)
print('fscore (Negative, Positive):', fscore)

precision (Negative, Positive): [0.67935578 0.76009792]
recall (Negative, Positive): [0.7030303  0.73928571]
fscore (Negative, Positive): [0.69099032 0.74954737]
